# goals
1. given a date or by default, the current date, create a '{date}.md' file with the full written date (ie. Thursday 9 November 2023) as the first line
2. define 3 sections for the file:
    1. general
        - quick notes + reminders
    2. scheduled
        - meetings
        - tasks + deadlines
    3. actual/notes
        - each meeting
        - each task (?)
            - this might not be useful to programmatically do. if i end up trying to build functionality to mark things as complete and have a program traverse the lines for completion markings to update corresponding tasks, then having multiple places where the tasks are might be an issue. but for workflow i do want the tasks to have a place in the notes so that i can leave info for myself from what i tried, etc. so. TODO: noodle more on how to include tasks in actual section of template

# Add scheduled meetings

- follows: https://learnpython.com/blog/working-with-icalendar-with-python/

In [7]:
# dependencies
from datetime import date, datetime
from zoneinfo import ZoneInfo
from icalendar import Calendar, Event, vCalAddress, vText

In [8]:
# support methods
def load_cal(icsname):
    with open(icsname, 'rb') as f:
        ecal = Calendar.from_ical(f.read())
    f.close()
    return ecal

In [9]:
ecal = load_cal("../frozen/Bailey Passmore_m.bailey.passmore@gmail.com.ics")

In [10]:
{component.name for component in ecal.walk()}

{'DAYLIGHT', 'STANDARD', 'VALARM', 'VCALENDAR', 'VEVENT', 'VTIMEZONE'}

In [23]:
events = {}
for event in ecal.walk("VEVENT"):
    if 'SUMMARY' in event:
        info = {
            'title': str(event['SUMMARY']),
            'timestamp': event.decoded('DTSTART'),
            'date': event.decoded('DTSTART').strftime('%Y-%m-%d'),
            'time': event.decoded('DTSTART').strftime('%H:%M'),
        }
        if 'LOCATION' in event: info['location'] = str(event['LOCATION'])
        if info['date'] not in events: events[info['date']] = [info]
        else: events[info['date']].append(info)

In [21]:
events

{'2016-06-03': [{'title': 'FALL REGISTRATION',
   'timestamp': datetime.date(2016, 6, 3),
   'date': '2016-06-03',
   'time': '00:00'}],
 '2016-08-15': [{'title': 'PHYS 208A (LEC)',
   'timestamp': datetime.datetime(2016, 8, 15, 10, 30, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
   'date': '2016-08-15',
   'time': '10:30',
   'location': 'Rm 2101'},
  {'title': 'MUSC 237',
   'timestamp': datetime.datetime(2016, 8, 15, 8, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
   'date': '2016-08-15',
   'time': '08:00',
   'location': 'Rm 6304'}],
 '2016-08-17': [{'title': 'PHYS 208A (LAB)',
   'timestamp': datetime.datetime(2016, 8, 17, 13, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
   'date': '2016-08-17',
   'time': '13:00',
   'location': 'RM 2101'}],
 '2016-08-19': [{'title': 'PHYS 208AX',
   'timestamp': datetime.datetime(2016, 8, 19, 8, 30, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
   'dat

In [18]:
for day, event_list in events.items():
    if '2023-11' in day:
        print(day)
        for event in event_list: print(f"{event['time']} - {event['title']}")
        print('\n')

2023-11-11
00:00 - oakland trip?
02:30 - ty
10:00 - wake + bake + make


2023-11-02
10:00 - MacArthur work sessions
13:00 - BP + LB: coworking time


2023-11-04
01:00 - Walk with Ty
20:00 - Protest for Ceasefire
19:00 - dropoff burrito?


2023-11-12
01:00 - Palestinian Rights Display


2023-11-20
17:00 - PB's Israel-Palestine Reading chat
02:00 - Teenage Halloween, with Blonde Star Wig, Dandelion Massacre, and Humidity Glow


2023-11-03
22:00 - makeup code review


2023-11-09
00:00 - Ty’s birthday


2023-11-10
21:00 - focus on a family: read/write about a family affected by police violence


2023-11-23
04:00 - The Emo Night Tour




In [30]:
recurs = {}
for event in ecal.walk("VEVENT"):
    if ('SUMMARY' in event):
        if ('TS + BP' in event['SUMMARY']):
            title = str(event['SUMMARY'])
            if title not in recurs: recurs[title] = [event] 
            else: recurs[title].append(event)

In [32]:
{k:len(v) for k,v in recurs.items()}

{'TS + BP': 1, 'TS + BP standing': 8}

In [41]:
sorted([event.decoded('DTSTART').strftime('%Y-%m-%d') 
 for event in recurs['TS + BP standing']])

['2023-02-01',
 '2023-02-15',
 '2023-03-01',
 '2023-03-06',
 '2023-03-29',
 '2023-04-05',
 '2023-04-19',
 '2023-07-12']

In [39]:
for event in recurs['TS + BP standing']:
    print(event, '\n\n')

VEVENT({'DTSTART': vDDDTypes(2023-02-01 13:00:00-08:00, Parameters({'TZID': 'America/Los_Angeles'})), 'DTEND': vDDDTypes(2023-02-01 14:30:00-08:00, Parameters({'TZID': 'America/Los_Angeles'})), 'RRULE': vRecur({'FREQ': ['WEEKLY'], 'UNTIL': [datetime.datetime(2023, 2, 15, 7, 59, 59, tzinfo=<UTC>)], 'BYDAY': ['WE']}), 'DTSTAMP': vDDDTypes(2023-11-14 18:00:11+00:00, Parameters({})), 'ORGANIZER': vCalAddress('b'mailto:11h4g90k8ln7g0v9cdc0h8bnak@group.calendar.google.com''), 'UID': vText('b'mjlptk1rtb86hnem4nplek4ang@google.com''), 'ATTENDEE': vCalAddress('b'mailto:m.bailey.passmore@gmail.com''), 'CREATED': vDDDTypes(2023-02-01 02:57:24+00:00, Parameters({})), 'DESCRIPTION': vText('b'Creating a recurring meeting to mark the time/link in our calendars.\xc2\xa0 Both of us have permissions to modify\\, cancel\\, update days/times\\, as necessary. <br><br>Join Zoom Meeting<br><a href="https://us02web.zoom.us/j/81720023961?pwd=S2pEcldLME9YYkk0QmJJZjVJNnBwdz09">https://us02web.zoom.us/j/817200239

# adding scheduled tasks

In [81]:
# dependencies
from pathlib import Path, PosixPath
from os.path import isdir, isfile
import pandas as pd

In [85]:
# support methods
def find_taskfiles(arg):
    if isfile(arg): return PosixPath(arg)
    if isdir(arg): return [path for path in Path(arg).rglob('*.parquet')]
    return None


def prep_tasks(arg):
    fs = find_taskfiles(arg)
    assert fs != []
    assert Path(fs[0]).exists()
    out = pd.concat([pd.read_parquet(f) for f in fs])
    return out

In [79]:
# main
taskdir = "/Users/home/git/my-TODO/tasks/"

In [89]:
prep_tasks(taskdir).sample(1).T

,22
source,/Users/home/git/my-TODO/individual/daily/20221...
filename,20221126.md
TODO_line,- TODO (reading) review notes from PB's MSE de...
tag,reading
timeline,None
task,review notes from PB's MSE demo [19 Oct 2022]
started,False
last_update,False
completed,False
